In [1]:
from utils.processor import *
from utils.solver import *
from config import *
from utils.tom_api import *

'''
Timer class - (SOURCE CODE) https://realpython.com/python-timer/
'''
import time

class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self._start_time = None

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = round(time.perf_counter() - self._start_time, 3)
        self._start_time = None
        print(f"Elapsed time: {elapsed_time} seconds")

timer = Timer()
timer.start()

In [4]:
data_dir = os.path.join(os.getcwd(), 'data')
datapath = os.path.join(data_dir, 'parking-meters.csv')
merged_datapath = os.path.join(data_dir, 'parking-meters_merged_twins.csv')
# origin = (HOME['lat'], HOME['lon'])
# dest = (PACIFIC_CENTER['lat'], PACIFIC_CENTER['lon'])
ttapi = TomSearchAPI(API_KEY)
dest = ttapi.find_location('Metrotown Chatime')
print(dest)
dest = dest['location']
dest = (dest['lat'], dest['lon'])
dest

{'poi_name': 'Chatime Metrotown', 'address': '5 Kingsway, Vancouver BC V5T 3H7', 'location': {'lat': 49.26446, 'lon': -123.10068}}


(49.26446, -123.10068)

In [82]:
topK = 10
stay_duration = 2 * 60 
parking_budget = 12.00

processor = Processor(datapath, merged_datapath, origin, dest, topK,
                      stay_duration, parking_budget, load_merged=True)
variables = processor.form_variables_dict()
variables

(3655, 24)
(3655, 25)
IDX - 0
Time limit: 2 	 Rate: 1.0
~~~~~~~~~~~~~~~~~~~~ Single ~~~~~~~~~~~~~~~~~~~~
*** Travel time: 27.6 ***
out: 2 - in: 1
~~~ Parking Status: 2 ~~~
{'b_1': 1.0, 'constraint_coeffs': {'parking_fee': 2.0, 'savings': 10.0, 'expected_avail': 2, 'et_next_out': 13.0, 'et_next_in': 22.0, 'walk_time': 5.0, 'travel_et': 27.6}, 'meterid': '280903', 'latln': (49.275647081858395, -123.11505965990052)}
IDX - 1
Time limit: 2 	 Rate: 1.0
******************** Twin **********************
*** Travel time: 27.633333333333333 ***
out: 2 - in: 1
~~~ Parking Status: 3 ~~~
{'b_2': 1.0, 'constraint_coeffs': {'parking_fee': 2.0, 'savings': 10.0, 'expected_avail': 3, 'et_next_out': 11.0, 'et_next_in': 23.0, 'walk_time': 5.167, 'travel_et': 27.633333333333333}, 'meterid': "('280907', '280905')", 'latln': (49.275559854939345, -123.11517762058094)}
IDX - 2
Time limit: 2 	 Rate: 1.0
******************** Twin **********************
*** Travel time: 27.683333333333334 ***
out: 2 - in: 1
~~~ Pa

{'y_1': {'b_1': 1.0,
  'constraint_coeffs': {'parking_fee': 2.0,
   'savings': 10.0,
   'expected_avail': 2,
   'et_next_out': 13.0,
   'et_next_in': 22.0,
   'walk_time': 5.0,
   'travel_et': 27.6},
  'meterid': '280903',
  'latln': (49.275647081858395, -123.11505965990052)},
 'y_2': {'b_2': 1.0,
  'constraint_coeffs': {'parking_fee': 2.0,
   'savings': 10.0,
   'expected_avail': 3,
   'et_next_out': 11.0,
   'et_next_in': 23.0,
   'walk_time': 5.167,
   'travel_et': 27.633333333333333},
  'meterid': "('280907', '280905')",
  'latln': (49.275559854939345, -123.11517762058094)},
 'y_3': {'b_3': 1.0,
  'constraint_coeffs': {'parking_fee': 2.0,
   'savings': 10.0,
   'expected_avail': 3,
   'et_next_out': 10.0,
   'et_next_in': 23.0,
   'walk_time': 5.317,
   'travel_et': 27.683333333333334},
  'meterid': "('280911', '280909')",
  'latln': (49.27547463882799, -123.11529056754179)},
 'y_4': {'b_4': 1.0,
  'constraint_coeffs': {'parking_fee': 2.0,
   'savings': 10.0,
   'expected_avail': 3

In [83]:
solver = Solver(variables)
solver.solve_LP()
soln = solver.get_soln_summary()
soln

ParkingOptimizer:
MINIMIZE
1.0*y_1 + 1.0*y_2 + 1.0*y_3 + 1.0*y_4 + 1.0*y_5 + 1.0*y_6 + 1.0*y_7 + 1.0*y_8 + 0.0
SUBJECT TO
_C1: 2 y_1 + 3 y_2 + 3 y_3 + 3 y_4 + 3 y_5 + 3 y_6 + 3 y_7 + 4 y_8 >= 3

_C2: 10 y_1 + 10 y_2 + 10 y_3 + 10 y_4 + 10 y_5 + 6 y_6 + 6 y_7 + 6 y_8 >= 8.5

_C3: - 27.6 y_1 - 27.6333333333 y_2 - 27.6833333333 y_3 - 26.5 y_4
 - 26.4833333333 y_5 - 26.3 y_6 - 26.2666666667 y_7 - 26.35 y_8
 >= -26.8520833333

_C4: - 5 y_1 - 5.167 y_2 - 5.317 y_3 - 5.45 y_4 - 5.683 y_5 - 3.783 y_6
 - 3.917 y_7 - 3.5 y_8 >= -4.727125

_C5: - 13 y_1 - 11 y_2 - 10 y_3 - 11 y_4 - 13 y_5 - 10 y_6 - 9 y_7 - 8 y_8
 >= -10.625

_C6: y_1 + y_2 + y_3 + y_4 + y_5 + y_6 + y_7 + y_8 = 1

VARIABLES
y_1 <= 1 Continuous
y_2 <= 1 Continuous
y_3 <= 1 Continuous
y_4 <= 1 Continuous
y_5 <= 1 Continuous
y_6 <= 1 Continuous
y_7 <= 1 Continuous
y_8 <= 1 Continuous

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ahnafayub/opt/anaconda3/envs/geodat/lib/python3.10/si

,var,optimal_weight,parking_fee,expected_avail,travel_et,walk_time,et_next_out,et_next_in
1,y_2,0.380772,2.0,3,27.633333,5.167,11.0,23.0
6,y_7,0.278497,6.0,3,26.266667,3.917,9.0,21.0
3,y_4,0.244228,2.0,3,26.500000,5.450,11.0,23.0
7,y_8,0.096503,6.0,4,26.350000,3.500,8.0,22.0
5,y_6,0.000000,6.0,3,26.300000,3.783,10.0,23.0
4,y_5,0.000000,2.0,3,26.483333,5.683,13.0,25.0
0,y_1,0.000000,2.0,2,27.600000,5.000,13.0,22.0
2,y_3,0.000000,2.0,3,27.683333,5.317,10.0,23.0


In [84]:
timer.stop()

Elapsed time: 20.056 seconds


# Visualization

In [85]:
import folium
from folium.features import DivIcon

fmap = folium.Map(location=dest, zoom_start=16, width="100%", height="50%")

# folium.Marker(
#     location=origin, icon=folium.Icon(color="red",icon="home", prefix='fa')
# ).add_to(fmap)

folium.Marker(
    location=dest, icon=folium.Icon(color="red",icon="location-dot", prefix='fa')
).add_to(fmap)


for i, spot in enumerate(soln.loc[soln.optimal_weight > 0]['var']):
    print(spot)
    rank = i+1
    spot_loc = variables[spot]['latln']
    folium.Marker(
        location = spot_loc,
        popup = f'Rank-{rank}',
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(7,20),
            html=f'<div style="font-size: 18pt; color : black">{rank}</div>',
            )
    ).add_to(fmap)
    fmap.add_child(folium.CircleMarker(spot_loc, radius=15))

fmap

y_2
y_7
y_4
y_8
